# Explorer les articles de Hacker News

Dans ce projet, nous comparerons deux types de messages différents de Hacker News, un site populaire où les histoires (publications) liées à la technologie sont votées et commentées. Les deux types de messages que nous explorerons commencent par Ask HN ou Show HN.

Les utilisateurs soumettent Ask HN posts pour poser une question spécifique à la communauté Hacker News, telle que "Quel est le meilleur cours en ligne que vous ayez jamais suivi?" De même, les utilisateurs soumettent des messages Show HN pour montrer à la communauté Hacker News un projet, un produit ou tout simplement quelque chose d'intéressant.

Nous comparerons spécifiquement ces deux types d'articles pour déterminer ce qui suit:
 - Est-ce que Ask HN ou Show HN reçoivent plus de commentaires en moyenne?
 - Les publications créées à un moment donné reçoivent-elles plus de commentaires en moyenne?
 
Il convient de noter que l'ensemble de données avec lequel nous travaillons a été réduit de près de 300 000 lignes à environ 20 000 lignes en supprimant toutes les soumissions qui n'ont reçu aucun commentaire, puis en échantillonnant au hasard les soumissions restantes.

# Introduction

Tout d'abord, nous allons lire les données et supprimer les en-têtes.

In [1]:
import csv

f = open('OneDrive\Documents\my_datasets\hacker_news.csv')
hn = list(csv.reader(f))
hn[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

In [2]:
header = hn[0]
hn = hn[1:]
print(hn[0])

['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


# Extraction de Ask HN et Show HN Posts

Tout d'abord, nous identifierons les publications qui commencent par Ask HN ou Show HN et séparerons les données de ces deux types de publications dans des listes différentes. La séparation des données facilite l'analyse des étapes suivantes.

In [3]:
ask_posts = []
show_posts = []
other_posts = []

In [4]:
for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

In [5]:
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


# Calcul du nombre moyen de commentaires pour Ask HN et Show HN Posts

In [6]:
total_ask_comments = 0
for row in ask_posts:
    num_com = int(row[4])
    total_ask_comments += num_com

avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)

14.038417431192661


In [7]:
total_show_comments = 0
for row in show_posts:
    num_com = int(row[4])
    total_show_comments += num_com

avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

10.31669535283993


En moyenne, Ask Posts de notre échantillon reçoit environ 14 commentaires, tandis que Show Posts en reçoit environ 10. Comme Ask Posts est plus susceptible de recevoir des commentaires, nous concentrerons notre analyse restante uniquement sur ces Posts.

# Recherche du nombre de Ask Posts et de commentaires par heure de création

Ensuite, nous déterminerons si nous pouvons maximiser la quantité de commentaires qu'un Ask Posts reçoit en le créant à un certain moment. Tout d'abord, nous trouverons le nombre de Ask Posts créés au cours de chaque heure de la journée, ainsi que le nombre de commentaires reçus par ces messages. Ensuite, nous calculerons le nombre moyen de commentaires Ask Posts créés à chaque heure de la journée de réception.

In [9]:
import datetime as dt

In [10]:
result_list = []
for row in ask_posts:
    result_list.append([row[6], int(row[4])])    

In [11]:
counts_by_hour = {}
comments_by_hour = {}
date_format = "%m/%d/%Y %H:%M"

In [46]:
for row in result_list:
    date = row[0]
    comment = row[1]
    time = dt.datetime.strptime(date, date_format).strftime("%H")
    if time in counts_by_hour:
        counts_by_hour[time] += 1
        comments_by_hour[time] += comment
    else:
        counts_by_hour[time] = 1
        comments_by_hour[time] = comment
comments_by_hour

{'09': 1004,
 '13': 5012,
 '10': 3172,
 '14': 5664,
 '16': 7256,
 '23': 2172,
 '12': 2748,
 '17': 4584,
 '15': 17908,
 '21': 6980,
 '20': 6888,
 '02': 5524,
 '18': 5756,
 '03': 1684,
 '05': 1856,
 '19': 4752,
 '01': 2732,
 '22': 1916,
 '08': 1968,
 '04': 1348,
 '00': 1788,
 '06': 1588,
 '07': 1068,
 '11': 2564}

# Calcul du nombre moyen de commentaires pour les messages Ask HN par heure

In [19]:
avg_by_hour = []
for hour in comments_by_hour:
    avg_by_hour.append([hour, comments_by_hour[hour]/counts_by_hour[hour]])

In [20]:
avg_by_hour

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

# Tri et impression de valeurs à partir d'une liste de listes

In [30]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

In [31]:
print(swap_avg_by_hour)

[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]


In [32]:
sorted_swap = sorted(swap_avg_by_hour, reverse = True)

In [34]:
print(sorted_swap[:5])

[[38.5948275862069, '15'], [23.810344827586206, '02'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21']]


In [43]:
time_format = "%H"
for row in sorted_swap[:5]:
    hour = dt.datetime.strptime(row[1], time_format).strftime("%H:00")
    average = row[0]
    template = "{h}: {avg:.2f} average comments per post".format(h=hour, avg=average)
    print(template)

15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


L'heure qui reçoit le plus de commentaires par publication en moyenne est 15h00, avec une moyenne de 38,59 commentaires par publication. Il y a une augmentation d'environ 60% du nombre de commentaires entre les heures où le nombre moyen de commentaires est le plus élevé et le deuxième plus élevé.

Selon la [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home) de l'ensemble de données, le fuseau horaire utilisé est l'heure de l'Est aux États-Unis. Ainsi, nous pourrions également écrire 15h00 comme ***3:00 pm est***.

# Conclusion

Dans ce projet, nous avons analysé ask_posts et show_posts pour déterminer quel type de message et quelle heure reçoivent le plus de commentaires en moyenne. Sur la base de notre analyse, pour maximiser le nombre de commentaires reçus par un message, ***nous vous recommandons de le classer dans la catégorie ask_post et de le créer entre 15 h 00 et 16 h 00*** (3:00 pm est - 4:00 pm est).